### 심층 신경망 과 최적화
1. 딥러닝 학습의 문제점화 해결방법
    1. 기울기 손실(Gradient Vanishing)
    1. 가중치 초기화
1. 최적화 알고
    

#### 기울기 손실(Gradient Vanishing)
더 깊은 Layer 에서는 더 학습이 잘되는거 아닌가? 하지만 기울기 손실이 발생한다.

기울기 손실이란? : Out에 나오는 값과 멀이질 수록 학습이 모호하게 진행됨 Sigmoid함수 에서의 1보다 작으면 0에 가까워진다는 이유로 계속 0에 가까워져 기울기가 모호해짐(역전파시 0에 수렴하는게 많음)
![gradient](https://user-images.githubusercontent.com/50629716/62521649-0536ce00-b86b-11e9-8737-0a84b67d27db.PNG)
* Sigmoid
도함수의 최대값은 0.25
* ReLU
도함수 값이 0이나 1이기 때문에 컴퓨터 측면에서 경제적 학습도 빠름
* Leaky ReLU
0보다 작은 경우 ReLU에서 신경이 죽어버리는 현상 극복
* Tanh
함수값의 범위가(-1,1), 도함수의 최댓값 1

#### 가중치 초기화
![sigmoid](https://user-images.githubusercontent.com/50629716/62523905-b2abe080-b86f-11e9-8c7c-a4cc098a9bf4.PNG)
* t = wx + b 에서 시그모이드 함수 미분시 t가 5만 넘어도 0에 수렴 -> 기울기 소실 문제 발생
* 가중치w를 0으로 초기화하면 두번째 층 레이어 의 뉴런이 모두 같은 값이 전달됨
![weight initalization](https://user-images.githubusercontent.com/50629716/62524407-86449400-b870-11e9-9296-59c74e95ab74.PNG)

**표준편차를 0.01로 하는 정규분포 형태로 초기화 했을 때 0.5 중심으로 모여있음.**

1. Xavier initalization (RBM보다 좋은 방법이 생김)
하나의 노드에 몇개의 입력이고 몇개의 출력인가에 맞게 비례해 초기값을 주는 방법.

    * 표준 정규 분포를 입력 개수의 제곤근으로 나누어줌.
    * W = np.random.randn(n_input,n_output)/sqrt(n_input)
![weight1](https://user-images.githubusercontent.com/50629716/62525026-a0cb3d00-b871-11e9-8ee7-3dcbff1350df.PNG)
![w2](https://user-images.githubusercontent.com/50629716/62525027-a0cb3d00-b871-11e9-90be-617c885ab639.PNG)


2. He initalization
ReLU 함수에 맞는 초기화 법
    * input 갯수의 절반의 제곱근으로 나누어 주면 된다.
    * W = np.random.randn(n_input,n_output)/sqrt(n_input/2)
https://gomguard.tistory.com/184

#### 최적화(Optimization) 알고리즘

1. SGD
    * 손실함수를 계산할 때 전체 Traing set을 사용함
    * 계산량이 너무 많아 지는것을 방지
    * 전체 데이터에서 일부 데이터의 모음에 대해서만 loss function을 계산
    * data를 sampling해서 gradient descent를 진행한다. 이를 반복

**loss function 재정리**

신경망을 학습할 때 학습 상태에 대해 측청하는 하나의 지표로 사용. 신경망의 가중치 매개변수들이 스스로 특징을 찾아 가기에 이 가중치 값이 최적이 될 수록 해야 하며 잘 찾아가고 있는지 볼때 손실함수를 본다.

1) Mean Squared Error(평균제곱오차)

    예측 값과 실제 값의 차이를 제곱하여 평균을 낸것
* 예측값과 실제값의 차이가 클 수록 평균 오차제곱도 커짐(작아야 좋음)
$$ E=\dfrac {1}{n}\sum ^{n}_{k}\left( y_{k}-t_{k}\right) ^{2} $$
    
2) Cross entropy error

    기본적으로 분류 문제에서 one hot 코딩을 했을 경우에만 사용할 수 있는 오차 계산 법
    밑이 자연로그인 e를 예측값에 씌워서 실제 값과 곱한후 전체 값을 합한후 음수로 변환
* 엔트로피가 낮을 수록 좋다
  $$  E=-\sum _{k}t_{k}\log y_{k} $$


#### 알고리즘 정리 사이트
http://shuuki4.github.io/deep%20learning/2016/05/20/Gradient-Descent-Algorithm-Overview.html
    
    
    

In [28]:
## SGD
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import optimizers

# 영화의 리뷰에 따라 데이터길이가 다르기 때문에 shape를 맞춰주어야 한다.
def sequence_shaping(sequence,dimension):
    results = np.zeros((len(sequence),dimension))
    for i, word_indices in enumerate(sequence): #enumerate 는 index와 그값 으로 이루어지게 하는
        results[i,word_indices] = 1.0 # 각 리뷰의 word_indices 위치에 1대입
    return results
# 100번째 까지 많이 사용하는 단어 추출
word_num = 100
data_num = 25000

(train_data, train_labels), (test_data, test_labels) = keras.datasets.imdb.load_data(num_words=word_num)
train_data = sequence_shaping(train_data, dimension = word_num)
test_data = sequence_shaping(test_data, dimension = word_num)

optimizer_model = keras.Sequential([
    keras.layers.Dense(128, activation = tf.nn.relu, input_shape =(word_num,)),
    keras.layers.Dense(128, activation = tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

optimizer_model.compile(optimizer=optimizers.Adam(lr=0.001,beta_1 = 0.05,beta_2 = 0.05),
                   loss = 'binary_crossentropy',
                   metrics = ['accuracy','binary_crossentropy'])
optimizer_model.summary()

optimizer_history = optimizer_model.fit(train_data,train_labels, epochs = 20, batch_size = 500, validation_data=(test_data, test_labels),verbose=2)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 128)               12928     
_________________________________________________________________
dense_30 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_31 (Dense)             (None, 1)                 129       
Total params: 29,569
Trainable params: 29,569
Non-trainable params: 0
_________________________________________________________________
Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 - 0s - loss: 0.6225 - acc: 0.6518 - binary_crossentropy: 0.6225 - val_loss: 0.5906 - val_acc: 0.6835 - val_binary_crossentropy: 0.5906
Epoch 2/20
25000/25000 - 0s - loss: 0.5835 - acc: 0.6902 - binary_crossentropy: 0.5835 - val_loss: 0.5851 - val_acc: 0.6924 - val_binary_crossentropy: 0.5851
Epoch 3/20
250